In [ ]:
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import itertools
from tqdm import tqdm
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

# Configuration
CONFIG_PATH = "data/config.json"
OUTPUT_DIR = "outputs"
NUM_RUNS = 10000
RANDOM_SEED = 42

# Create sample config if needed
def create_sample_config():
    return {
        "credits_start": 100,
        "spins_per_round": 8,
        "max_rounds": 3,
        "initial_bar_target": 1.0,
        "reels": {
            "rows": 1,
            "cols": 3,
            "symbols": ["A", "B", "C", "D", "E"],
            "multipliers": {"A": 2, "B": 3, "C": 4, "D": 5, "E": 6},
            "probabilities": {"A": 0.2, "B": 0.2, "C": 0.2, "D": 0.2, "E": 0.2}
        },
        "bar_fill_per_match": {"3_same": 0.20, "2_same": 0.067, "1_same": 0},
        "bar_bonus_multiplier": 2.0,
        "upgrades": {
            "reel_bias": {"cost": 50, "effect": "increase the probability of D by 5% and E by 10%, decrease the probability of A by 10% and B by 5%"},
            "extra_spins": {"cost": 50, "effect": "+2 spins per round"},
            "bonus_multiplier_upgrade": {"cost": 50, "effect": "increase all symbol multipliers by 1.0"}
        }
    }

# Load or create config
try:
    with open(CONFIG_PATH, "r") as f:
        config = json.load(f)
except FileNotFoundError:
    os.makedirs(os.path.dirname(CONFIG_PATH), exist_ok=True)
    config = create_sample_config()
    with open(CONFIG_PATH, "w") as f:
        json.dump(config, f, indent=2)

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Game engine
class SlotGame:
    def __init__(self, config_dict):
        self.config = config_dict
        self.credits = float(self.config.get("credits_start", 100))
        self.spins_per_round = int(self.config.get("spins_per_round", 8))
        
        reels_config = self.config["reels"]
        self.cols = int(reels_config.get("cols", 3))
        self.symbols = list(reels_config["symbols"])
        self.base_multipliers = {s: float(v) for s, v in reels_config["multipliers"].items()}
        
        raw_probs = {s: float(reels_config["probabilities"].get(s, 0.0)) for s in self.symbols}
        total_prob = sum(raw_probs.values())
        self.base_symbol_probs = {s: raw_probs[s] / total_prob for s in raw_probs}
        
        self.bar_fill_per_match = self.config.get("bar_fill_per_match", {"3_same": 0.1, "2_same": 0.05, "1_same": 0})
        self.base_bar_bonus_multiplier = float(self.config.get("bar_bonus_multiplier", 2.0))
        
        self.bar_progress = 0.0
        self.bar_target = float(self.config.get("initial_bar_target", 1.0))
        self.round = 1
        self.max_rounds = int(self.config.get("max_rounds", 3))
        
        self.upgrades = {"reel_bias": False, "extra_spins": False, "bonus_multiplier_upgrade": False}
        self.upgrade_bought_this_round = False
    
    def get_current_probabilities(self):
        probs = self.base_symbol_probs.copy()
        if self.upgrades.get("reel_bias"):
            if "D" in probs: probs["D"] += 0.05
            if "E" in probs: probs["E"] += 0.10
            if "A" in probs: probs["A"] = max(0.0, probs["A"] - 0.10)
            if "B" in probs: probs["B"] = max(0.0, probs["B"] - 0.05)
            total_prob = sum(probs.values())
            probs = {s: probs[s] / total_prob for s in probs}
        return probs
    
    def get_effective_multipliers(self):
        if self.upgrades["bonus_multiplier_upgrade"]:
            return {s: val + 1.0 for s, val in self.base_multipliers.items()}
        return dict(self.base_multipliers)
    
    def spin_reels(self):
        current_probs = self.get_current_probabilities()
        symbols = list(current_probs.keys())
        weights = list(current_probs.values())
        return [random.choices(symbols, weights=weights, k=1)[0] for _ in range(self.cols)]
    
    def evaluate_spin(self, row):
        symbol_counts = {s: row.count(s) for s in self.symbols}
        bar_filled_this_spin = 0.0
        matched = False
        multipliers = self.get_effective_multipliers()
        
        for symbol, count in symbol_counts.items():
            if count == 3:
                increment = multipliers[symbol] * float(self.bar_fill_per_match.get("3_same", 0.0))
                self.bar_progress += increment
                bar_filled_this_spin += increment
                matched = True
            elif count == 2:
                increment = multipliers[symbol] * float(self.bar_fill_per_match.get("2_same", 0.0))
                self.bar_progress += increment
                bar_filled_this_spin += increment
                matched = True
        
        if not matched:
            singles = [s for s, c in symbol_counts.items() if c == 1]
            if singles and float(self.bar_fill_per_match.get("1_same", 0.0)) > 0:
                best_symbol = max(singles, key=lambda s: multipliers[s])
                increment = multipliers[best_symbol] * float(self.bar_fill_per_match.get("1_same", 0.0))
                self.bar_progress += increment
                bar_filled_this_spin += increment
        
        if self.bar_progress > self.bar_target:
            self.bar_progress = self.bar_target
        
        return 0.0, bar_filled_this_spin
    
    def buy_upgrade(self, upgrade_name):
        if self.upgrade_bought_this_round:
            return "You can only buy one upgrade per round."
        upgrade_config = self.config.get("upgrades", {})
        if upgrade_name not in upgrade_config:
            return f"Upgrade '{upgrade_name}' does not exist."
        if self.upgrades.get(upgrade_name):
            return f"Upgrade '{upgrade_name}' already purchased."
        cost = float(upgrade_config[upgrade_name]["cost"])
        if self.credits < cost:
            return f"Not enough credits to buy {upgrade_name}"
        self.credits -= cost
        self.upgrades[upgrade_name] = True
        self.upgrade_bought_this_round = True
        return f"Upgrade {upgrade_name} purchased successfully."
    
    def play_round_silent(self):
        self.bar_progress = 0.0
        self.upgrade_bought_this_round = False
        spins_this_round = self.spins_per_round + (2 if self.upgrades["extra_spins"] else 0)
        
        for spin_num in range(1, spins_this_round + 1):
            row = self.spin_reels()
            payout, filled = self.evaluate_spin(row)
        
        bonus_triggered = False
        if self.bar_progress >= self.bar_target:
            multiplier = float(self.base_bar_bonus_multiplier)
            self.credits *= multiplier
            bonus_triggered = True
        
        return bonus_triggered

# Utils
def generate_upgrade_strategies():
    upgrades = ["reel_bias", "extra_spins", "bonus_multiplier_upgrade"]
    strategies = [("skip", "skip")]
    for u in upgrades:
        strategies.append(("skip", u))
        strategies.append((u, "skip"))
    for u1 in upgrades:
        for u2 in upgrades:
            if u1 != u2:
                strategies.append((u1, u2))
    return strategies

def format_strategy_name(strategy_tuple):
    round2, round3 = strategy_tuple
    if round2 == "skip" and round3 == "skip":
        return "No Upgrades"
    elif round2 == "skip":
        return f"R3: {round3}"
    elif round3 == "skip":
        return f"R2: {round2}"
    else:
        return f"R2: {round2} → R3: {round3}"

def format_upgrade_combo_name(upgrade_list):
    if not upgrade_list or all(u == "skip" for u in upgrade_list):
        return "No Upgrades"
    active_upgrades = [u for u in upgrade_list if u != "skip"]
    if not active_upgrades:
        return "No Upgrades"
    return " + ".join(sorted(active_upgrades))

# Statistical analysis

def generate_reel_spin_stats(config):
    """Generate detailed reel spin statistics"""
    
    # Create game instances for all combinations
    base_game = SlotGame(config_dict=config)
    
    reel_bias_game = SlotGame(config_dict=config)
    reel_bias_game.upgrades["reel_bias"] = True
    
    bonus_mult_game = SlotGame(config_dict=config)
    bonus_mult_game.upgrades["bonus_multiplier_upgrade"] = True
    
    both_upgrades_game = SlotGame(config_dict=config)
    both_upgrades_game.upgrades["reel_bias"] = True
    both_upgrades_game.upgrades["bonus_multiplier_upgrade"] = True
    
    symbols = base_game.symbols
    base_probs = base_game.get_current_probabilities()
    reel_bias_probs = reel_bias_game.get_current_probabilities()
    base_multipliers = base_game.get_effective_multipliers()
    bonus_multipliers = bonus_mult_game.get_effective_multipliers()
    
    results = []
    
    # Generate all possible combinations
    for row in itertools.product(symbols, repeat=base_game.cols):
        outcome_str = " ".join(row)
        
        # Calculate probabilities
        base_prob = 1.0
        for sym in row:
            base_prob *= base_probs[sym]
        
        bias_prob = 1.0
        for sym in row:
            bias_prob *= reel_bias_probs[sym]
        
        symbol_counts = {s: row.count(s) for s in symbols}
        
        # Classify outcome type
        outcome_type = "1_same"
        max_count = max(symbol_counts.values())
        if max_count == 3:
            outcome_type = "3_same"
        elif max_count == 2:
            outcome_type = "2_same"
        
        # Calculate base barfill (no upgrades)
        base_barfill = 0.0
        for symbol, count in symbol_counts.items():
            if count == 3:
                base_barfill += base_multipliers[symbol] * base_game.bar_fill_per_match.get("3_same", 0.0)
            elif count == 2:
                base_barfill += base_multipliers[symbol] * base_game.bar_fill_per_match.get("2_same", 0.0)
        
        if base_barfill == 0.0 and base_game.bar_fill_per_match.get("1_same", 0.0) > 0:
            singles = [s for s, c in symbol_counts.items() if c == 1]
            if singles:
                best_symbol = max(singles, key=lambda s: base_multipliers[s])
                base_barfill = base_multipliers[best_symbol] * base_game.bar_fill_per_match.get("1_same", 0.0)
        
        # Calculate barfill with bonus multiplier upgrade only
        bonus_barfill = 0.0
        for symbol, count in symbol_counts.items():
            if count == 3:
                bonus_barfill += bonus_multipliers[symbol] * base_game.bar_fill_per_match.get("3_same", 0.0)
            elif count == 2:
                bonus_barfill += bonus_multipliers[symbol] * base_game.bar_fill_per_match.get("2_same", 0.0)
        
        if bonus_barfill == 0.0 and base_game.bar_fill_per_match.get("1_same", 0.0) > 0:
            singles = [s for s, c in symbol_counts.items() if c == 1]
            if singles:
                best_symbol = max(singles, key=lambda s: bonus_multipliers[s])
                bonus_barfill = bonus_multipliers[best_symbol] * base_game.bar_fill_per_match.get("1_same", 0.0)
    
        
        results.append({
            "Reel spin outcome": outcome_str,
            "Outcome type": outcome_type,
            "Probability": base_prob,
            "Probability with reel_bias upgrade": bias_prob,
            "Barfill": base_barfill,
            "Barfill with Bonus Multiplier upgrade": bonus_barfill,
        })
    
    return pd.DataFrame(results)


def bar_fill_distribution(game):
    symbols = game.symbols
    multipliers = game.get_effective_multipliers()
    current_probs = game.get_current_probabilities()
    values, probs = [], []
    
    for row in itertools.product(symbols, repeat=game.cols):
        p_row = 1.0
        for sym in row:
            p_row *= current_probs[sym]
        
        symbol_counts = {s: row.count(s) for s in symbols}
        increment = 0.0
        
        for symbol, count in symbol_counts.items():
            if count == 3:
                increment += multipliers[symbol] * game.bar_fill_per_match.get("3_same", 0.0)
            elif count == 2:
                increment += multipliers[symbol] * game.bar_fill_per_match.get("2_same", 0.0)
        
        if increment == 0.0 and game.bar_fill_per_match.get("1_same", 0.0) > 0:
            singles = [s for s, c in symbol_counts.items() if c == 1]
            if singles:
                best_symbol = max(singles, key=lambda s: multipliers[s])
                increment = multipliers[best_symbol] * game.bar_fill_per_match.get("1_same", 0.0)
        
        values.append(increment)
        probs.append(p_row)
    
    return np.array(values), np.array(probs)

def expected_stats(game):
    values, probs = bar_fill_distribution(game)
    exp = np.sum(values * probs)
    exp_sq = np.sum((values ** 2) * probs)
    var = exp_sq - exp ** 2
    std = np.sqrt(max(var, 0.0))
    expected_bonus_contribution = exp * game.base_bar_bonus_multiplier
    return exp, std, expected_bonus_contribution

def calculate_round_success_probability(game, bar_target, num_spins):
    """
    Calculate the probability that bar_progress >= bar_target after num_spins
    using dynamic programming for efficiency.
    """
    values, probs = bar_fill_distribution(game)
    
    # dp[spin][progress] = probability of having exactly 'progress' after 'spin' spins
    # Discretize progress to avoid infinite states
    
    # Find maximum possible progress and discretize
    max_single_fill = max(values)
    max_total_fill = max_single_fill * num_spins
    
    # Use small step size for accuracy
    step_size = 0.001
    max_steps = int(max_total_fill / step_size) + 1
    
    # Initialize DP table
    # dp[progress_step] = probability of having exactly that progress
    current_dp = {0: 1.0}  # Start with 0 progress, probability 1
    
    for spin in range(num_spins):
        next_dp = {}
        
        for current_progress_step, current_prob in current_dp.items():
            if current_prob == 0:
                continue
                
            current_progress = current_progress_step * step_size
            
            for fill_value, fill_prob in zip(values, probs):
                new_progress = min(current_progress + fill_value, bar_target)
                new_progress_step = int(new_progress / step_size)
                
                if new_progress_step not in next_dp:
                    next_dp[new_progress_step] = 0.0
                next_dp[new_progress_step] += current_prob * fill_prob
        
        current_dp = next_dp
    
    # Sum probabilities for all states where progress >= bar_target
    success_prob = 0.0
    target_step = int(bar_target / step_size)
    
    for progress_step, prob in current_dp.items():
        if progress_step >= target_step:
            success_prob += prob
    
    return min(success_prob, 1.0)  # Ensure probability doesn't exceed 1

def calculate_strategy_success_probabilities(config, strategy_tuple):
    """
    Calculate success probabilities for all three rounds of a strategy.
    """
    round2_upgrade, round3_upgrade = strategy_tuple
    bar_targets = [1.0, 1.5, 2.0]
    base_spins = config.get("spins_per_round", 8)
    
    results = {}
    
    # Round 1: Base game only
    game_r1 = SlotGame(config_dict=config)
    spins_r1 = base_spins
    p1 = calculate_round_success_probability(game_r1, bar_targets[0], spins_r1)
    results['round_1'] = p1
    
    # Round 2: Base game + Round 2 upgrade
    game_r2 = SlotGame(config_dict=config)
    if round2_upgrade != "skip":
        game_r2.upgrades[round2_upgrade] = True
    spins_r2 = base_spins + (2 if game_r2.upgrades["extra_spins"] else 0)
    p2 = calculate_round_success_probability(game_r2, bar_targets[1], spins_r2)
    results['round_2'] = p2
    
    # Round 3: Base game + both upgrades
    game_r3 = SlotGame(config_dict=config)
    if round2_upgrade != "skip":
        game_r3.upgrades[round2_upgrade] = True
    if round3_upgrade != "skip":
        game_r3.upgrades[round3_upgrade] = True
    spins_r3 = base_spins + (2 if game_r3.upgrades["extra_spins"] else 0)
    p3 = calculate_round_success_probability(game_r3, bar_targets[2], spins_r3)
    results['round_3'] = p3
    
    return results

def generate_strategy_summary(config):
    strategies = generate_upgrade_strategies()
    summary_rows = []
    
    for strategy_tuple in strategies:
        game = SlotGame(config_dict=config)
        round2_upgrade, round3_upgrade = strategy_tuple
        if round2_upgrade != "skip":
            game.upgrades[round2_upgrade] = True
        if round3_upgrade != "skip":
            game.upgrades[round3_upgrade] = True
        
        exp_fill, std_fill, exp_bonus_contrib = expected_stats(game)
        current_probs = game.get_current_probabilities()
        multipliers = game.get_effective_multipliers()
        spins_per_round = game.spins_per_round + (2 if game.upgrades["extra_spins"] else 0)
        expected_per_round = exp_fill * spins_per_round
        
        # Calculate actual success probabilities
        success_probs = calculate_strategy_success_probabilities(config, strategy_tuple)
        p1, p2, p3 = success_probs['round_1'], success_probs['round_2'], success_probs['round_3']
        
        # Calculate upgrade costs
        upgrade_costs = {}
        upgrade_costs['round_2'] = 0
        upgrade_costs['round_3'] = 0
        
        if round2_upgrade != "skip":
            upgrade_costs['round_2'] = float(config.get("upgrades", {}).get(round2_upgrade, {}).get("cost", 0))
        if round3_upgrade != "skip":
            upgrade_costs['round_3'] = float(config.get("upgrades", {}).get(round3_upgrade, {}).get("cost", 0))
        
        total_upgrade_cost = upgrade_costs['round_2'] + upgrade_costs['round_3']
        
        # Calculate expected final credits using probabilistic approach
        starting_credits = float(config.get("credits_start", 100))
        bar_multiplier = float(config.get("bar_bonus_multiplier", 2.0))
        
        # Expected credits calculation accounting for all possible outcomes
        # Outcome 1: Fail Round 1 (probability: 1-p1)
        # Credits: starting_credits
        outcome_1_prob = (1 - p1)
        outcome_1_credits = starting_credits
        
        # Outcome 2: Pass R1, Fail R2 (probability: p1 * (1-p2))
        # Credits: starting_credits * multiplier - r2_upgrade_cost
        outcome_2_prob = p1 * (1 - p2)
        outcome_2_credits = starting_credits * bar_multiplier - upgrade_costs['round_2']
        
        # Outcome 3: Pass R1&R2, Fail R3 (probability: p1 * p2 * (1-p3))
        # Credits: starting_credits * multiplier^2 - r2_cost - r3_cost
        outcome_3_prob = p1 * p2 * (1 - p3)
        outcome_3_credits = starting_credits * (bar_multiplier ** 2) - upgrade_costs['round_2'] - upgrade_costs['round_3']
        
        # Outcome 4: Pass All Rounds (probability: p1 * p2 * p3)
        # Credits: starting_credits * multiplier^3 - r2_cost - r3_cost
        outcome_4_prob = p1 * p2 * p3
        outcome_4_credits = starting_credits * (bar_multiplier ** 3) - upgrade_costs['round_2'] - upgrade_costs['round_3']
        
        # Expected final credits
        expected_final_credits = (
            outcome_1_prob * outcome_1_credits +
            outcome_2_prob * outcome_2_credits +
            outcome_3_prob * outcome_3_credits +
            outcome_4_prob * outcome_4_credits)
        
        # Expected ROI
        expected_roi = (expected_final_credits - starting_credits) / starting_credits if starting_credits > 0 else 0.0
        
        summary_rows.append({
            "Strategy": format_strategy_name(strategy_tuple),
            "Round_2_Upgrade": round2_upgrade,
            "Round_3_Upgrade": round3_upgrade,
            "Upgrade_Combination": format_upgrade_combo_name(list(strategy_tuple)),
            "Expected_Fill_per_Spin": exp_fill,
            "Std_Dev_Fill": std_fill,
            "Expected_Bonus_Contribution": exp_bonus_contrib,
            "Spins_per_Round": spins_per_round,
            "Expected_Fill_per_Round": expected_per_round,
            "R1_Success_Probability": p1,
            "R2_Success_Probability": p2,
            "R3_Success_Probability": p3,
            "Overall_Success_Probability": p1 * p2 * p3,
            "Total_Upgrade_Cost": total_upgrade_cost,
            "Avg_Symbol_Multiplier": sum(multipliers.values()) / len(multipliers),
            "Expected_ROI": expected_roi,
            "Expected_Final_Credits": expected_final_credits
        })
    
    return pd.DataFrame(summary_rows)

# Simulation
def run_single_game(config_dict, strategy_tuple):
    game = SlotGame(config_dict=config_dict)
    total_spins = 0
    rounds_played = 0
    upgrade_costs_paid = 0
    upgrades_purchased = []
    
    while game.round <= game.max_rounds:
        spins_this_round = game.spins_per_round + (2 if game.upgrades["extra_spins"] else 0)
        
        if game.round in (2, 3):
            choice = strategy_tuple[game.round - 2]
            if choice != "skip" and not game.upgrades.get(choice, False):
                upgrade_config = config_dict.get("upgrades", {})
                if choice in upgrade_config:
                    cost = float(upgrade_config[choice]["cost"])
                    if game.credits >= cost:
                        result = game.buy_upgrade(choice)
                        if "purchased successfully" in result:
                            upgrade_costs_paid += cost
                            upgrades_purchased.append(choice)
        
        bonus = game.play_round_silent()
        total_spins += spins_this_round
        rounds_played = game.round
        
        if not bonus:
            break
        
        if game.round < game.max_rounds:
            game.bar_target += 0.5
            game.round += 1
        else:
            break
    
    return {
        "final_credits": game.credits,
        "rounds_played": rounds_played,
        "total_spins": total_spins,
        "upgrade_costs_paid": upgrade_costs_paid,
        "upgrades_purchased": upgrades_purchased,
        "strategy": format_strategy_name(strategy_tuple)
    }

def simulate_strategy(config_dict, strategy_tuple, num_runs=10000, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    
    results = []
    for i in range(num_runs):
        res = run_single_game(config_dict, strategy_tuple)
        results.append(res)
    
    df = pd.DataFrame(results)
    stats = {
        "strategy": format_strategy_name(strategy_tuple),
        "round2_upgrade": strategy_tuple[0],
        "round3_upgrade": strategy_tuple[1], 
        "upgrade_combination": format_upgrade_combo_name(list(strategy_tuple)),
        "num_runs": num_runs,
        "avg_final_credits": df["final_credits"].mean(),
        "std_final_credits": df["final_credits"].std(),
        "median_final_credits": df["final_credits"].median(),
        "min_final_credits": df["final_credits"].min(),
        "max_final_credits": df["final_credits"].max(),
        "avg_rounds_played": df["rounds_played"].mean(),
        "completion_rate": (df["rounds_played"] == 3).mean(),
        "avg_upgrade_costs": df["upgrade_costs_paid"].mean(),
        "net_credits_gained": df["final_credits"].mean() - 100,
        "roi": (df["final_credits"].mean() - 100) / 100,
    }
    return stats, df

# Run analysis
print("Running complete slot game analysis...")
strategies = generate_upgrade_strategies()

# Theoretical analysis
print("1. Theoretical analysis...")
theoretical_df = generate_strategy_summary(config)

# Simulations
print("2. Monte Carlo simulations...")
all_simulation_results = []
all_detailed_results = []

for i, strategy_tuple in enumerate(tqdm(strategies, desc="Simulating")):
    summary_stats, detailed_df = simulate_strategy(config, strategy_tuple, num_runs=NUM_RUNS, seed=RANDOM_SEED + i)
    all_simulation_results.append(summary_stats)
    detailed_df["strategy"] = summary_stats["strategy"]
    detailed_df["strategy_tuple"] = str(strategy_tuple)
    all_detailed_results.append(detailed_df)

simulation_df = pd.DataFrame(all_simulation_results)
detailed_simulation_df = pd.concat(all_detailed_results, ignore_index=True)

# Create comparison dataframe
print("3. Creating comparison analysis...")
comparison_df = theoretical_df.merge(
    simulation_df[['strategy', 'completion_rate', 'roi', 'avg_final_credits', 'avg_rounds_played']], 
    left_on='Strategy', 
    right_on='strategy', 
    how='left'
).drop('strategy', axis=1)

# Add RTP calculations for easier comparison
comparison_df['Theoretical_RTP'] = comparison_df['Expected_ROI'].fillna(0)
comparison_df['Simulation_RTP'] = comparison_df['roi'].fillna(0)

# Save results
print("4. Saving results...")
with pd.ExcelWriter(os.path.join(OUTPUT_DIR, "analysis_results.xlsx"), engine="openpyxl") as writer:
    theoretical_df.to_excel(writer, sheet_name="Theoretical", index=False)
    simulation_df.to_excel(writer, sheet_name="Simulation", index=False)
    comparison_df.to_excel(writer, sheet_name="Comparison", index=False)

# Generate reel spin stats 
print("5. Generating reel spin statistics...")
reel_stats_df = generate_reel_spin_stats(config)
reel_stats_df.to_excel(os.path.join(OUTPUT_DIR, "reel_spin_stats.xlsx"), index=False)
print(f"Reel spin stats saved to: {OUTPUT_DIR}/reel_spin_stats.xlsx")

# Summary
print(f"\nResults saved to: {OUTPUT_DIR}/analysis_results.xlsx")
print(f"Total strategies: {len(comparison_df)}")
print(f"Strategies with >50% overall success: {(comparison_df['Overall_Success_Probability'] > 0.5).sum()}")
print(f"Average simulation completion rate: {comparison_df['completion_rate'].mean():.1%}")
print(f"Best strategy (by simulation): {comparison_df.loc[comparison_df['completion_rate'].idxmax(), 'Strategy']}")
print(f"Best completion rate: {comparison_df['completion_rate'].max():.1%}")

# Display top results with comparison
print("\nTop 5 strategies by simulation completion rate:")
top_strategies = comparison_df.nlargest(5, 'completion_rate')[
    ['Strategy', 'completion_rate', 'Simulation_RTP', 'Theoretical_RTP']
]
print(top_strategies.to_string(index=False))

print("\nTop 5 strategies by theoretical RTP:")
top_theoretical = comparison_df.nlargest(5, 'Theoretical_RTP')[
    ['Strategy', 'Theoretical_RTP', 'Simulation_RTP', 'completion_rate'] 
]
print(top_theoretical.to_string(index=False))

print("\nAnalysis complete")


Running complete slot game analysis...
1. Theoretical analysis...
2. Monte Carlo simulations...


Simulating: 100%|██████████| 13/13 [00:08<00:00,  1.53it/s]

3. Creating comparison analysis...
4. Saving results...
5. Generating reel spin statistics...
Reel spin stats saved to: outputs/reel_spin_stats.xlsx

Results saved to: outputs/analysis_results.xlsx
Total strategies: 13
Strategies with >50% overall success: 0
Average simulation completion rate: 32.3%
Best strategy (by simulation): R2: reel_bias
Best completion rate: 41.3%

Top 5 strategies by simulation completion rate:
                                    Strategy  completion_rate  Simulation_RTP  Theoretical_RTP
                               R2: reel_bias           0.4133         1.40510         1.732822
             R2: reel_bias → R3: extra_spins           0.4102         1.34120         1.895748
R2: reel_bias → R3: bonus_multiplier_upgrade           0.4043         1.27490         1.821470
                             R2: extra_spins           0.3600         1.16400         1.422178
                R2: bonus_multiplier_upgrade           0.3578         1.17535         1.424686

Top 5 